## Glazing Branches Order Efficiency

### Importing Required  Packages

In [1]:
import pygsheets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
#sns.set(style='ticks', palette='Set2')
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import pytz
import businesstimedelta
import holidays as pyholidays
%matplotlib inline
import os
import requests
import json
#pip3 install --upgrade pip
#pip3 install jupyter

today = date.today()
last=today-relativedelta(months=4)
last=str(last)
today=str(today)

### Login to the APA DW

In [2]:
#Login
url = "http://10.10.15.222/api/v1/auth/login"

payload = json.dumps({
 "email": "derick@optica.africa",
  "password":os.environ.get("API_PASSWORD")
})

headers = {
 'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
result = response.json()

access_token = result["access_token"]

###  Pulling Required Data Sets--OrderScreenc1 and OrderScreen

In [3]:
####Screen log/Workload/Order Screen Child
#######################################################################################################

url = "http://10.10.15.222/api/v1/fetch/OrderScreenc1"
today = date.today()
last=today-relativedelta(months=3)
last=str(last)
today=str(today)


payload = json.dumps({
  "entity": "OrderScreenc1",
  "filter_column": "Date",
  "start":last,
  "end": today
})


headers = {'Authorization': f"Bearer {access_token}",'Content-Type': 'application/json'}

response = requests.request("PUT", url, headers=headers, data=payload)

result = response.json()
import pandas as pd
df= pd.DataFrame(result["data"])

Workload=df[["odsc_createdby","Date","Time","Status","DocEntry","odsc_remarks"]].rename(
      columns={"odsc_createdby":"Created User","odsc_remarks":"TO_VARCHAR(U_VSPRMKS)"})

Workload["Date"]=pd.to_datetime(Workload["Date"]).dt.date
Workload["Time"]=pd.to_datetime(Workload["Time"],format= '%H%M').dt.time
MasterData=Workload
MasterData["TimeStamp"] = pd.to_datetime(MasterData.Date.astype(str) + ' ' + MasterData.Time.astype(str), format="%Y%m%d %H:%M:%S",errors="coerce")

In [4]:
####Orders/Order Screen
#######################################################################################################
today = date.today()
last=today-relativedelta(months=6)
last=str(last)
today=str(today)
url = "http://10.10.15.222/api/v1/fetch/OrderScreen"

payload = json.dumps({
  "entity": "OrderScreen",
  "filter_column": "Creation_Time",
  "start": last,
  "end": today
})


headers = {'Authorization': f"Bearer {access_token}",'Content-Type': 'application/json'}

response = requests.request("PUT", url, headers=headers, data=payload)

result = response.json()
import pandas as pd
df= pd.DataFrame(result["data"])

AllOrders =df.rename(columns={"doc_entry":"DocEntry","doc_no":"Order No.","ods_creator":"Order Creator","cust_code":"Customer Code",
                       "ods_ordertype":"Order Type","ods_status":"Status","ods_status1":"Status1",
                       "ods_outlet":"Outlet","ods_orderno":"Sales Order No.","ods_orderprocess_branch":"OrderProcess Branch",
                             "ods_ordercriteriastatus":"OrderCriteria"})


AllOrders_data=AllOrders[["DocEntry","Order No.","OrderProcess Branch","OrderCriteria","Outlet","ods_repair_branch","ods_preqc_tech","ods_final_qc_tech","Status"]]

### Defining/Filtering various statuses Timestamps i.e salesorder created,awaitting for collection,collected etc

In [5]:

#############################################################################################
##Sales Orders
salesCreatedStatus=['Sales Order Created','Confirmed by Approvals Team']
SalesOrderCreated=MasterData[MasterData.Status.isin(salesCreatedStatus)]
SalesOrderCreated=SalesOrderCreated[["DocEntry","TimeStamp"]]
SalesOrderCreated.rename(columns={"TimeStamp":"SalesOrderCreated"},inplace=True)

################################################################################################
##Awaitting for collection Data

AwaittingForCollection=MasterData[MasterData.Status=="Awaiting for Collection"]
AwaittingForCollection.rename(columns={"TimeStamp":"Awaitting_For_Collection"},inplace=True)
###############################################################################################
##Collected
Collected=MasterData[MasterData.Status=="Collected"]
Collected=Collected[["DocEntry","TimeStamp"]]
Collected.rename(columns={"TimeStamp":"Collected"},inplace=True)
SalesOrderCreated=pd.merge(SalesOrderCreated,Collected,on="DocEntry",how="outer")
########################################Time from Satelite Branch to Glazing Branch##################################################################################
##Satelite Branches sending Orders
Sending_Status = ["Printed PF Identifier","Printed Frame Identifier","Printed Repair Identifier","Printed Frame Identifier","Printed Lens Identifier",
          "Printed PF to Follow Identifier","Printed Frame and Lens Identifier"]

Sending_Orders=MasterData[MasterData.Status.isin(Sending_Status)]
Sending_Orders=Sending_Orders[["DocEntry","TimeStamp"]]
Sending_Orders.rename(columns={"TimeStamp":"Satelite_Sending"},inplace=True)
##Main Branches Receiving Orders
Status_Receiving= ["Order Printed at Branch","Order Printed","Assigned to Technician at Branch"]
Receiving_Orders=MasterData[MasterData.Status.isin(Status_Receiving)]
Receiving_Orders=Receiving_Orders[["DocEntry","TimeStamp"]]
Receiving_Orders.rename(columns={"TimeStamp":"Main_Receiving"},inplace=True)
SateliteBranch_Sending_Time=pd.merge(Sending_Orders,Receiving_Orders,on="DocEntry",how="outer")

#####################################Time from Glazing Branch to Satellite#####################################################################################
Sending_Orders=MasterData[MasterData.Status=="Fitting Process Completed at Branch"]
Sending_Orders=Sending_Orders[["DocEntry","TimeStamp"]]
Sending_Orders.rename(columns={"TimeStamp":"Main_Sending"},inplace=True)

Receiving_Orders=MasterData[MasterData.Status=="Awaiting for Collection"]
Receiving_Orders=Receiving_Orders[["DocEntry","TimeStamp"]]
Receiving_Orders.rename(columns={"TimeStamp":"Satelite_Receiving"},inplace=True)
MainBranch_Sending_Time=pd.merge(Sending_Orders,Receiving_Orders,on="DocEntry",how="outer")


Satelite_MainBranch_RiderTimings=pd.merge(SateliteBranch_Sending_Time,MainBranch_Sending_Time,on="DocEntry",how="outer")

SalesOrderCreated=pd.merge(SalesOrderCreated,Satelite_MainBranch_RiderTimings,on="DocEntry",how="outer")


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### Defining Your Date Range for the Report

In [6]:
MasterData=Workload
unique=MasterData['Date'].unique()
unique=pd.DataFrame(unique)
unique=pd.to_datetime(unique[0],infer_datetime_format=True)
print(f"Enter Date Range, starting with 'start date' to 'end date' comma separated {unique}" )
From,To =str(input()).split(",")

Enter Date Range, starting with 'start date' to 'end date' comma separated 0    2022-03-23
1    2022-03-24
2    2022-03-25
3    2022-03-26
4    2022-03-27
        ...    
85   2022-03-18
86   2022-03-19
87   2022-03-20
88   2022-03-21
89   2022-03-22
Name: 0, Length: 90, dtype: datetime64[ns]
2022-03-21,2022-03-26


### Filtering the Data as per the Defined Date Range

In [7]:
AwaittingForCollection["From"]=From
AwaittingForCollection["To"]=To
AwaittingForCollection["To"]= pd.to_datetime(AwaittingForCollection["To"].astype(str), format="%Y-%m-%d",errors="coerce")
AwaittingForCollection["From"]= pd.to_datetime(AwaittingForCollection["From"].astype(str), format="%Y-%m-%d",errors="coerce")
AwaittingForCollection["Comparison"]= pd.to_datetime(AwaittingForCollection["Date"].astype(str), format="%Y-%m-%d",errors="coerce")

AwaittingForCollection =AwaittingForCollection[(AwaittingForCollection["Comparison"] >=AwaittingForCollection["From"]) &
                                              (AwaittingForCollection["Comparison"] <=AwaittingForCollection["To"])] 

AwaittingForCollection=AwaittingForCollection[["DocEntry","Awaitting_For_Collection"]]





C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3857814921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwaittingForCollection["From"]=From
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3857814921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwaittingForCollection["To"]=To
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3857814921.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

### Glazing Branches Data

In [8]:
data=pd.merge(AwaittingForCollection,SalesOrderCreated,on="DocEntry",how="left")
GlazingBranchesData=pd.merge(AllOrders_data,Workload,on="DocEntry",how="right")
FilterStatus=['Fitting Process Completed at Branch','Sent to Packaging']
FilterProcessingBranch=['','0LE','0MA','0VE','0DS','HQWS']
Criterias=["Branch Frame and HQ Lens Glazed at Branch","PF and HQ Lens Glazed at Branch","Surfacing Lens with Branch Frame Glazed at Branch","Surfacing Lens with PF Glazed at Branch"]
GlazingBranchesData=GlazingBranchesData[(~GlazingBranchesData["OrderProcess Branch"].isin(FilterProcessingBranch)) &
                                        (GlazingBranchesData["Status_y"].isin(FilterStatus))&
                                        (GlazingBranchesData["Status_x"]!='Cancel')&
                                        (~GlazingBranchesData["OrderCriteria"].isin(Criterias))]

In [9]:

GlazingBranchesData=pd.merge(GlazingBranchesData,data,on="DocEntry",how="inner")
GlazingBranchesData.drop(["Status_x","Date","Time","Status_y","TO_VARCHAR(U_VSPRMKS)","TimeStamp","ods_repair_branch","ods_preqc_tech"],
                        inplace=True,axis=1)


### NaKuru and its Satelite Branches 


In [11]:
Nakuru_Branches=["NAK","NWE","GOL"]
Nakuru_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(Nakuru_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(18),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=Nakuru_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=Nakuru_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=Nakuru_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=Nakuru_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)

####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=Nakuru_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=Nakuru_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=Nakuru_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=Nakuru_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
#######################################################
Nakuru_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60 
Nakuru_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60 
Nakuru_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs)*60 
Nakuru_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs)*60 



C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\workalendar\core.py:436: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3427328183.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nakuru_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3427328183.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

### Eldoret and its Satelite Branches 

In [12]:
Eldoret_Branches=["ELD","RUP","HIG"]
Eldoret_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(Eldoret_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(18),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=Eldoret_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=Eldoret_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=Eldoret_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=Eldoret_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=Eldoret_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=Eldoret_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=Eldoret_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=Eldoret_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
#######################################################
Eldoret_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
Eldoret_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60 
Eldoret_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs)*60 
Eldoret_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs)*60 

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2633932179.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Eldoret_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2633932179.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Eldoret_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2633932179.py:46: SettingWithCopyWarning: 
A value is trying to be set on a cop

### Thika and its Satelite Branches 

In [13]:
Thika_Branches=["THI","TBZ"]
Thika_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(Thika_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(8,30),end_time=datetime.time(17,30),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=Thika_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=Thika_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=Thika_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=Thika_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(15),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=Thika_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=Thika_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=Thika_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=Thika_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
#######################################################
Thika_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60  
Thika_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60 
Thika_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs)*60 
Thika_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs)*60 

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3080755141.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thika_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3080755141.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Thika_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3080755141.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of

### Kisumu and its Satelite Branches 

In [14]:
Kisumu_Branches=["KIS","KWE","TUF","UNI"]
Kisumu_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(Kisumu_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(18),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=Kisumu_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=Kisumu_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=Kisumu_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=Kisumu_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=Kisumu_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=Kisumu_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=Kisumu_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=Kisumu_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
#######################################################
Kisumu_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60 
Kisumu_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60 
Kisumu_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs)*60 
Kisumu_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs)*60 

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3318987173.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kisumu_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3318987173.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kisumu_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3318987173.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy 

### Mombasa and its Satelite Branches 

In [15]:
Mombasa_Branches=["MSA","NYA","LIK"]
Mombasa_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(Mombasa_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(18),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=Mombasa_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=Mombasa_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=Mombasa_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=Mombasa_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(9),end_time=datetime.time(16),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=Mombasa_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=Mombasa_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=Mombasa_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=Mombasa_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)

#######################################################
Mombasa_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60 
Mombasa_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60 
Mombasa_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs)*60 
Mombasa_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs)*60 

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2234125780.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mombasa_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2234125780.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Mombasa_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2234125780.py:47: SettingWithCopyWarning: 
A value is trying to be set on a cop

### TRM and its Satelite Branches

In [16]:
TRM_Branches=["TRM","GAR"]
TRM_Data=GlazingBranchesData[GlazingBranchesData["OrderProcess Branch"].isin(TRM_Branches)]

####Days of the week
workday = businesstimedelta.WorkDayRule(start_time=datetime.time(10),end_time=datetime.time(19),working_days=[0,1, 2, 3, 4])
vic_holidays = pyholidays.KE()
holidays = businesstimedelta.HolidayRule(vic_holidays)
from workalendar.africa import Kenya
cal = Kenya()
hl = cal.holidays()
my_dict=dict(hl)
vic_holidays=vic_holidays.append(my_dict)
businesshrs = businesstimedelta.Rules([workday, holidays])

def BusHrs(start, end):
    if end>=start:
        return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""
    
SalesToAwCollectn_Wkhrs=TRM_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Wkhrs=TRM_Data.apply(lambda row: BusHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Wkhrs=TRM_Data.apply(lambda row: BusHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Wkhrs=TRM_Data.apply(lambda row: BusHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
####Weekends
# Define a working weekend day(Saturday)

Saturday = businesstimedelta.WorkDayRule(start_time=datetime.time(10),end_time=datetime.time(18,30),working_days=[5])

businesshrs = businesstimedelta.Rules([Saturday, holidays])

def SatHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sathrs=TRM_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sathrs=TRM_Data.apply(lambda row: SatHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sathrs=TRM_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sathrs=TRM_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)


# Define a working weekend day(Sunday)

Sunday = businesstimedelta.WorkDayRule(start_time=datetime.time(11),end_time=datetime.time(17),working_days=[6])

businesshrs = businesstimedelta.Rules([Sunday, holidays])

def SunHrs(start, end):
    if end>=start:
         return businesshrs.difference(start,end).hours+float(businesshrs.difference(start,end).seconds)/float(3600)
    else:
        return ""

    
SalesToAwCollectn_Sunhrs=TRM_Data.apply(lambda row: SunHrs(row["SalesOrderCreated"], row['Awaitting_For_Collection']), axis=1)
SalesToCollectn_Sunhrs=TRM_Data.apply(lambda row: SunHrs(row["SalesOrderCreated"], row['Collected']), axis=1)
SateliteSending_Sunhrs=TRM_Data.apply(lambda row: SatHrs(row["Satelite_Sending"], row['Main_Receiving']), axis=1)
SateliteReceiving_Sunhrs=TRM_Data.apply(lambda row: SatHrs(row["Main_Sending"], row['Awaitting_For_Collection']), axis=1)
#######################################################
TRM_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs+SalesToAwCollectn_Sunhrs)*60
TRM_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs+SalesToCollectn_Sunhrs)*60 
TRM_Data["SateliteSendingTime"]=(SateliteSending_Wkhrs+SateliteSending_Sathrs+SateliteSending_Sunhrs)*60 
TRM_Data["SateliteReceivingTime"]=(SateliteReceiving_Wkhrs+SateliteReceiving_Sathrs+SateliteReceiving_Sunhrs)*60 

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3855198092.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRM_Data["Duration"]=(SalesToAwCollectn_Wkhrs+SalesToAwCollectn_Sathrs+SalesToAwCollectn_Sunhrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3855198092.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRM_Data["Sales_To_Collected"]=(SalesToCollectn_Wkhrs+SalesToCollectn_Sathrs+SalesToCollectn_Sunhrs)*60
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3855198092.py:65: SettingWithCopyWarnin

In [17]:
Final_GlazingBranchesData=pd.concat([Nakuru_Data,Eldoret_Data,Thika_Data,Mombasa_Data,Kisumu_Data,TRM_Data])
GlazingBranchesData[~GlazingBranchesData["DocEntry"].isin(Final_GlazingBranchesData["DocEntry"])]
Final_GlazingBranchesData["Duration"]=pd.to_numeric(Final_GlazingBranchesData["Duration"])
Final_GlazingBranchesData["Sales_To_Collected"]=pd.to_numeric(Final_GlazingBranchesData["Sales_To_Collected"])
Final_GlazingBranchesData["SateliteSendingTime"]=pd.to_numeric(Final_GlazingBranchesData["SateliteSendingTime"])
Final_GlazingBranchesData["SateliteReceivingTime"]=pd.to_numeric(Final_GlazingBranchesData["SateliteReceivingTime"])


### Grouping the Duration Taken

In [18]:

Main_GlazingBranchesData=Final_GlazingBranchesData[Final_GlazingBranchesData["Outlet"].isin(["KIS","ELD","NAK","THI","TRM","MSA"])]
Satelite_GlazingBranchesData=Final_GlazingBranchesData[~Final_GlazingBranchesData["Outlet"].isin(["KIS","ELD","NAK","THI","TRM","MSA"])]

###Sales Created to Awaiting for Collection
Main_GlazingBranchesData["Category"]=np.where(Main_GlazingBranchesData.Duration<=15,"Ready Within 15 Min",
                                              np.where(Main_GlazingBranchesData.Duration<=30,"Ready btwn 16 - 30 Min",
                                                      "Greater than 30mins"))
                                                   
Satelite_GlazingBranchesData["Category"]=np.where(Satelite_GlazingBranchesData.Duration<=60,"Ready Within 1hr",
                                              np.where(Satelite_GlazingBranchesData.Duration<=120,"Ready btwn >1 -2hrs",
                                                       np.where(Satelite_GlazingBranchesData.Duration<=180,"Ready btwn >2 -3hrs",
                                                                np.where(Satelite_GlazingBranchesData.Duration<=240,"Ready btwn >3 -4hrs",
                                                    
                                                      "Greater than 4hrs"))))

###Sales Created to  Collection

Main_GlazingBranchesData["Category(collected)"]=np.where(Main_GlazingBranchesData.Sales_To_Collected.isnull(),"Not Yet Collected",
                                                         np.where(Main_GlazingBranchesData.Sales_To_Collected<=15,"Ready Within 15 Min",
                                              np.where(Main_GlazingBranchesData.Sales_To_Collected<=30,"Ready btwn 16 - 30 Min",
                                                      "Greater than 30mins")))
                                                   
Satelite_GlazingBranchesData["Category(collected)"]=np.where(Satelite_GlazingBranchesData.Sales_To_Collected.isnull(),"Not Yet Collected",
                                                             np.where(Satelite_GlazingBranchesData.Sales_To_Collected<=60,"Ready Within 1hr",
                                              np.where(Satelite_GlazingBranchesData.Sales_To_Collected<=120,"Ready btwn >1 -2hrs",
                                                       np.where(Satelite_GlazingBranchesData.Sales_To_Collected<=180,"Ready btwn >2 -3hrs",
                                                                np.where(Satelite_GlazingBranchesData.Sales_To_Collected<=240,"Ready btwn >3 -4hrs",
                                                                         "Greater than 4hrs")))))


##########################################################################################
###Satelite Branches Sending the Orders
data=Final_GlazingBranchesData
data["Category(Sending)"]=np.where(data.SateliteSendingTime.isnull(),"Not Yet",
                                                       np.where(data.SateliteSendingTime<=30,"Arrived within 30 Min",
                                                                np.where(data.SateliteSendingTime<=60,"Arrived >30 to 60 Min",
                                                                         np.where(data.SateliteSendingTime<=120,"Arrived >1hr to 2hr",
                                                                                  "Arrived in >2hr"))))

###Satelite Branches Receiving the Orders

data["Category(Receiving)"]=np.where(data.SateliteReceivingTime.isnull(),"Not Yet",
                                                         np.where(data.SateliteReceivingTime<=30,"Received within 30 Min",
                                                                  np.where(data.SateliteReceivingTime<=60,"Received >30 to 60 Min",
                                                                           np.where(data.SateliteReceivingTime<=120,"Received >1hr to 2hr",
                                                                                    "Received in >2hr"))))
                                                       
                                                   


                                                   

C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3122629012.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_GlazingBranchesData["Category"]=np.where(Main_GlazingBranchesData.Duration<=15,"Ready Within 15 Min",
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3122629012.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Satelite_GlazingBranchesData["Category"]=np.where(Satelite_GlazingBranchesData.Duration<=60,"Ready Within 1hr",
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3122629012.py:18: SettingW

### Aggregations via Pivot tables

### A. Sales Order Created to  Awaiting for Collection and to Finaly Collected

#### 1. Main Branches

In [19]:
###Sales Created to Awaiting for Collection
Main_Branches_Pivot=pd.pivot_table(Main_GlazingBranchesData,index="Outlet",columns="Category",values="Duration",
                                   aggfunc=["count","mean"],margins=True)

Main_Branches_Pivot_Counts=Main_Branches_Pivot[["count"]]
Main_Branches_Pivot_Counts.columns = Main_Branches_Pivot_Counts.columns.droplevel(0)
Main_Branches_Pivot_mean=Main_Branches_Pivot[["mean"]]
Main_Branches_Pivot_mean.columns = Main_Branches_Pivot_mean.columns.droplevel(0)
Main_Branches_Pivot_mean["Metric"]="Average Time (mins)"
Main_Branches_Pivot_Counts["Metric"]="Order Count"
Main_Branches_Pivot_mean=Main_Branches_Pivot_mean[["Metric","Ready Within 15 Min","Ready btwn 16 - 30 Min","Greater than 30mins",
                                                  "All"]]

Main_Branches_Pivot_Counts=Main_Branches_Pivot_Counts[["Metric","Ready Within 15 Min","Ready btwn 16 - 30 Min","Greater than 30mins",
                                                  "All"]]

Main_Branches_Pivot_Counts["%_ge Ready Within 30mins"]=(((Main_Branches_Pivot_Counts["Ready Within 15 Min"])+(Main_Branches_Pivot_Counts["Ready btwn 16 - 30 Min"]))/Main_Branches_Pivot_Counts["All"])*100

Final_Main_Branches_Pivot=pd.concat([Main_Branches_Pivot_mean,Main_Branches_Pivot_Counts])
Final_Main_Branches_Pivot=Final_Main_Branches_Pivot.reset_index()

###Sales Created to  Collection

Main_Branches_Pivot2=pd.pivot_table(Main_GlazingBranchesData,index="Outlet",columns="Category(collected)",values="Sales_To_Collected",
                                   aggfunc=["count","mean"],margins=True)

Main_Branches_Pivot2_Counts=Main_Branches_Pivot2[["count"]]
Main_Branches_Pivot2_Counts.columns = Main_Branches_Pivot2_Counts.columns.droplevel(0)
Main_Branches_Pivot2_mean=Main_Branches_Pivot2[["mean"]]
Main_Branches_Pivot2_mean.columns = Main_Branches_Pivot2_mean.columns.droplevel(0)
Main_Branches_Pivot2_mean["Metric"]="Average Time (mins)"
Main_Branches_Pivot2_Counts["Metric"]="Order Count"
Main_Branches_Pivot2_mean=Main_Branches_Pivot2_mean[["Metric","Ready Within 15 Min","Ready btwn 16 - 30 Min","Greater than 30mins",
                                                  "All"]]

Main_Branches_Pivot2_Counts=Main_Branches_Pivot2_Counts[["Metric","Ready Within 15 Min","Ready btwn 16 - 30 Min","Greater than 30mins",
                                                  "All"]]

Main_Branches_Pivot2_Counts["%_ge Ready Within 30mins"]=(((Main_Branches_Pivot2_Counts["Ready Within 15 Min"])+(Main_Branches_Pivot2_Counts["Ready btwn 16 - 30 Min"]))/Main_Branches_Pivot2_Counts["All"])*100

Final_Main_Branches_Pivot2=pd.concat([Main_Branches_Pivot2_mean,Main_Branches_Pivot2_Counts])
Final_Main_Branches_Pivot2=Final_Main_Branches_Pivot2.reset_index()


C:\Users\HP\AppData\Local\Temp/ipykernel_12032/1641852220.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_Branches_Pivot_mean["Metric"]="Average Time (mins)"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/1641852220.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_Branches_Pivot_Counts["Metric"]="Order Count"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/1641852220.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

#### 2. Satelite  Branches

In [20]:
###Sales Created to Awaiting for Collection
Satelite_Branches_Pivot=pd.pivot_table(Satelite_GlazingBranchesData,index="Outlet",columns="Category",values="Duration",
                                   aggfunc=["count","mean"],margins=True)

Satelite_Branches_Pivot_Counts=Satelite_Branches_Pivot[["count"]]
Satelite_Branches_Pivot_Counts.columns = Satelite_Branches_Pivot_Counts.columns.droplevel(0)
Satelite_Branches_Pivot_mean=Satelite_Branches_Pivot[["mean"]]
Satelite_Branches_Pivot_mean.columns = Satelite_Branches_Pivot_mean.columns.droplevel(0)
Satelite_Branches_Pivot_mean["Metric"]="Average Time (mins)"
Satelite_Branches_Pivot_Counts["Metric"]="Order Count"
Satelite_Branches_Pivot_mean=Satelite_Branches_Pivot_mean[["Metric","Ready Within 1hr","Ready btwn >1 -2hrs","Ready btwn >1 -2hrs","Ready btwn >2 -3hrs",
                                                  "Ready btwn >3 -4hrs","Ready btwn >3 -4hrs","Greater than 4hrs","All"]]

Satelite_Branches_Pivot_Counts=Satelite_Branches_Pivot_Counts[["Metric","Ready Within 1hr","Ready btwn >1 -2hrs","Ready btwn >1 -2hrs","Ready btwn >2 -3hrs",
                                                  "Ready btwn >3 -4hrs","Ready btwn >3 -4hrs","Greater than 4hrs","All"]]

#Satelite_Branches_Pivot_Counts["%_ge Ready Within 1hr"]=(Satelite_Branches_Pivot_Counts["Ready Within 1hr"]/Satelite_Branches_Pivot_Counts["All"])*100
Final_Satelite_Branches_Pivot=pd.concat([Satelite_Branches_Pivot_mean,Satelite_Branches_Pivot_Counts])
Final_Satelite_Branches_Pivot=Final_Satelite_Branches_Pivot.reset_index()
Final_Satelite_Branches_Pivot

###Sales Created to  Collection

Satelite_Branches_Pivot2=pd.pivot_table(Satelite_GlazingBranchesData,index="Outlet",columns="Category(collected)",values="Sales_To_Collected",
                                   aggfunc=["count","mean"],margins=True)

Satelite_Branches_Pivot2_Counts=Satelite_Branches_Pivot2[["count"]]
Satelite_Branches_Pivot2_Counts.columns = Satelite_Branches_Pivot2_Counts.columns.droplevel(0)
Satelite_Branches_Pivot2_mean=Satelite_Branches_Pivot2[["mean"]]
Satelite_Branches_Pivot2_mean.columns = Satelite_Branches_Pivot2_mean.columns.droplevel(0)
Satelite_Branches_Pivot2_mean["Metric"]="Average Time (mins)"
Satelite_Branches_Pivot2_Counts["Metric"]="Order Count"
Satelite_Branches_Pivot2_mean=Satelite_Branches_Pivot2_mean[["Metric","Ready Within 1hr","Ready btwn >1 -2hrs","Ready btwn >1 -2hrs","Ready btwn >2 -3hrs",
                                                  "Ready btwn >3 -4hrs","Ready btwn >3 -4hrs","Greater than 4hrs","All"]]

Satelite_Branches_Pivot2_Counts=Satelite_Branches_Pivot2_Counts[["Metric","Ready Within 1hr","Ready btwn >1 -2hrs","Ready btwn >1 -2hrs","Ready btwn >2 -3hrs",
                                                  "Ready btwn >3 -4hrs","Ready btwn >3 -4hrs","Greater than 4hrs","All"]]

#Satelite_Branches_Pivot_Counts["%_ge Ready Within 1hr"]=(Satelite_Branches_Pivot_Counts["Ready Within 1hr"]/Satelite_Branches_Pivot_Counts["All"])*100
Final_Satelite_Branches_Pivot2=pd.concat([Satelite_Branches_Pivot2_mean,Satelite_Branches_Pivot2_Counts])
Final_Satelite_Branches_Pivot2=Final_Satelite_Branches_Pivot2.reset_index()





C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2279123528.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Satelite_Branches_Pivot_mean["Metric"]="Average Time (mins)"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2279123528.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Satelite_Branches_Pivot_Counts["Metric"]="Order Count"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/2279123528.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [21]:
data["Category(Sending)"].unique()

array(['Arrived within 30 Min', 'Arrived >30 to 60 Min',
       'Arrived >1hr to 2hr', 'Arrived in >2hr', 'Not Yet'], dtype=object)

### B. Satelite Branches Sending Orders and Receiving Orders

In [22]:
###Satelite Branches Sending Orders
Main_Branches_Pivot=pd.pivot_table(data,index="Outlet",columns="Category(Sending)",values="SateliteSendingTime",
                                   aggfunc=["count","mean"],margins=True)

Main_Branches_Pivot_Counts=Main_Branches_Pivot[["count"]]
Main_Branches_Pivot_Counts.columns = Main_Branches_Pivot_Counts.columns.droplevel(0)
Main_Branches_Pivot_mean=Main_Branches_Pivot[["mean"]]
Main_Branches_Pivot_mean.columns = Main_Branches_Pivot_mean.columns.droplevel(0)
Main_Branches_Pivot_mean["Metric"]="Average Time (mins)"
Main_Branches_Pivot_Counts["Metric"]="Order Count"
# Main_Branches_Pivot_mean=Main_Branches_Pivot_mean[["Metric","Arrived within 30 Min","Arrived >30 to 60 Min","Arrived >1hr to 2hr",
#                                                   "Arrived in >2hr","All"]]

# Main_Branches_Pivot_Counts=Main_Branches_Pivot_Counts[["Metric","Arrived within 30 Min","Arrived >30 to 60 Min","Arrived >1hr to 2hr",
#                                                   "Arrived in >2hr","All"]]

Final_SateliteBranchesSending=pd.concat([Main_Branches_Pivot_mean,Main_Branches_Pivot_Counts])
Final_SateliteBranchesSending=Final_SateliteBranchesSending.reset_index()

###Satelite Branches Receiving Orders

Main_Branches_Pivot=pd.pivot_table(data,index="Outlet",columns="Category(Receiving)",values="SateliteReceivingTime",
                                   aggfunc=["count","mean"],margins=True)

Main_Branches_Pivot_Counts=Main_Branches_Pivot[["count"]]
Main_Branches_Pivot_Counts.columns = Main_Branches_Pivot_Counts.columns.droplevel(0)
Main_Branches_Pivot_mean=Main_Branches_Pivot[["mean"]]
Main_Branches_Pivot_mean.columns = Main_Branches_Pivot_mean.columns.droplevel(0)
Main_Branches_Pivot_mean["Metric"]="Average Time (mins)"
Main_Branches_Pivot_Counts["Metric"]="Order Count"
# Main_Branches_Pivot_mean=Main_Branches_Pivot_mean[["Metric","Arrived within 30 Min","Arrived >30 to 60 Min","Arrived >1hr to 2hr",
#                                                   "Arrived in >2hr","All"]]

# Main_Branches_Pivot_Counts=Main_Branches_Pivot_Counts[["Metric","Arrived within 30 Min","Arrived >30 to 60 Min","Arrived >1hr to 2hr",
#                                                   "Arrived in >2hr","All"]]

Final_SateliteBranchesReceiving=pd.concat([Main_Branches_Pivot_mean,Main_Branches_Pivot_Counts])
Final_SateliteBranchesReceiving=Final_SateliteBranchesReceiving.reset_index()





C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3841032386.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_Branches_Pivot_mean["Metric"]="Average Time (mins)"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3841032386.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_Branches_Pivot_Counts["Metric"]="Order Count"
C:\Users\HP\AppData\Local\Temp/ipykernel_12032/3841032386.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

### Writing to excel file time taken from Sales order created to Awaiting for Collection and final Collected.

In [23]:
#!pip install xlsxwriter --upgrade

import xlsxwriter
print(xlsxwriter.__version__)


#Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\SalesOrderToAwaitingCollected_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        Final_Main_Branches_Pivot.to_excel(writer,sheet_name="MainBranches_AwatnCollection", index=False)
        Final_Satelite_Branches_Pivot.to_excel(writer,sheet_name="SateliteBranches_AwtnCollection", index=False)
        for group, dataframe in Final_Main_Branches_Pivot.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            dataframe.to_excel(writer,sheet_name=name, index=False)
        for group, dataframe in Final_Satelite_Branches_Pivot.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            dataframe.to_excel(writer,sheet_name=name, index=False)
        
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()
######################################################################################################

with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\SalesOrderToCollected_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        Final_Main_Branches_Pivot2.to_excel(writer,sheet_name="MainBranches_Collection", index=False)
        Final_Satelite_Branches_Pivot2.to_excel(writer,sheet_name="SateliteBranches_Collection", index=False)
        for group, dataframe in Final_Main_Branches_Pivot2.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            #dataframe=dataframe.transpose()
            dataframe.to_excel(writer,sheet_name=name, index=False)
        for group, dataframe in Final_Satelite_Branches_Pivot2.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            #dataframe=dataframe.transpose()
            dataframe.to_excel(writer,sheet_name=name, index=False)
        
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

3.0.2


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### Writing to excel file time taken for Satelite Branches to Sent and Receive Orders

In [24]:
Final_SateliteBranchesSending=Final_SateliteBranchesSending[~Final_SateliteBranchesSending.Outlet.isin(["KIS","ELD","NAK","THI","TRM","MSA"])]
with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\SateliteSending_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        for group, dataframe in Final_SateliteBranchesSending.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            dataframe.to_excel(writer,sheet_name=name, index=False)
            
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()
        
##########################################################################################
Final_SateliteBranchesReceiving=Final_SateliteBranchesReceiving[~Final_SateliteBranchesReceiving.Outlet.isin(["KIS","ELD","NAK","THI","TRM","MSA"])]
with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\SateliteReceiving_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        for group, dataframe in Final_SateliteBranchesReceiving.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            dataframe.to_excel(writer,sheet_name=name, index=False)
            
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

### Writting to excel Detailes Data and Also the into excel sheets per branch for the orders that took more than 30 mins

In [25]:
#detailed_data=pd.concat([Main_GlazingBranchesData,Satelite_GlazingBranchesData])
with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\Details_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        Final_GlazingBranchesData.to_excel(writer,sheet_name="Details", index=False)
              
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()
        
######################################################################################################################
Morethan30mins=Main_GlazingBranchesData[["Order No.","OrderProcess Branch","OrderCriteria","Outlet","SalesOrderCreated","Awaitting_For_Collection","Duration"]]
Morethan30min=Morethan30mins[Morethan30mins.Duration>30]
        
with pd.ExcelWriter(r"C:\Users\HP\Documents\Master\Reports\Wairimu Mathenge\Glazing Branches\TookMorethan30mins_GlazingReport.xlsx", engine='xlsxwriter') as writer:    
        for group, dataframe in Morethan30min.groupby('Outlet'):
            # save the dataframe for each group to a csv
            dataframe = dataframe.drop(["Outlet"],axis=1)
            name = f'{group}'
            dataframe.to_excel(writer,sheet_name=name, index=False)
            
writer.save()

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
